In [18]:
# Data
from datetime import date
from random import randint
import pandas as pd

num = [randint(0, 100) for i in range(100)]
df = pd.DataFrame(num, columns=["count"])
df['dates'] = [date(2014+i, 1, 1) for i in range(100)]
df['grp'] = ['A' if x > 80 else 'B' if  x > 30 else 'C' for x in num]
df['item1'] = [randint(0, 100) for i in range(100)]
df['item2'] = [randint(0, 100) for i in range(100)]
df['item3'] = [randint(0, 100) for i in range(100)]

df.head(3)

,count,dates,grp,item1,item2,item3
0,20,2014-01-01,C,49,97,1
1,93,2015-01-01,A,21,56,33
2,4,2016-01-01,C,4,91,99


In [27]:
df2=df
df2['d']=pd.to_datetime(df2['dates'])
df2.head(2)

,count,dates,grp,item1,item2,item3,d
0,20,2014-01-01,C,49,97,1,2014-01-01
1,93,2015-01-01,A,21,56,33,2015-01-01


In [55]:
#- Adjust a plot based on range widget
# WIP

from bokeh.layouts import column
from bokeh.plotting import figure, output_file, show ,output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import RangeSlider
from bokeh.models.callbacks import CustomJS
output_notebook()

df['dates'] = pd.to_datetime(df['dates'])
df.sort('count', ascending = False)
source = ColumnDataSource(df)

#p = figure(plot_width=500, plot_height=300, logo=None, toolbar_location='right', x_axis_type="datetime") 
p = figure(plot_width=500, plot_height=300, logo=None, toolbar_location='right') 
p.circle(source=source, x='count', y='item1')  

callback = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var original_data = source.data;
#        var f = cb_obj.range
        var f0 = cb_obj.start
        var f1 = cb_obj.end
        x = data['count']
        y = data['item1']
        for (var key in original_data) {
            data[key] = [];
            for (i = 0; i < x.length; i++) {
#                if(x[i]>f[0] && x[i]<f[1]){
                 if(x[i]>f0 && x[i]<f1){
                    data[key].push(original_data[key][i]);
                }
            }
        }
target_obj.trigger('change');
source.trigger('change');
""")


_min = int(min(s.data['count']))
_max = int(max(s.data['count']))

range_slider = RangeSlider(start=0, end=100, value=(0,100), step=10, title="Range")
range_slider.js_on_change('value', callback)

l = layout([range_slider, p])
show(l)

Loading BokehJS ...

/Users/tomas/miniconda2/envs/py27_nb/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [ ]:
#- time series analysis
# http://localhost:8888/notebooks/timeseries/AnalyticsVidija/Time_Series_AirPassenger.ipynb



In [ ]:
# https://github.com/patricknieto/dashboard/blob/master/test2.py
# WIP, only the last df and tab is modified
#
import pandas as pd
import numpy as np

from bokeh.io import curdoc,  output_file, show
from bokeh.layouts import row, column, widgetbox, layout
from bokeh.models import ColumnDataSource, DateFormatter
from bokeh.models.widgets import Slider, TextInput, Panel, Tabs, CheckboxGroup, Div
from bokeh.models.widgets import Toggle, DataTable, DateFormatter, TableColumn, Button
from bokeh.plotting import figure,output_notebook,output_file
from bokeh.models.callbacks import CustomJS



output_file('test2.html')
#output_notebook()

files = ['urlvisits-20160823.csv', 'searches-20160823.csv', 'uniqueusers-20160823.csv', 'intab-20160823.csv','urlvisits-20160823.csv']
names = ['URL Visits', 'Searches', 'Unique Users', 'In-Tab Users','Test']


def create_panels(files, names):

    tab_list = []

    for i in range(len(files)):

        df = pd.read_csv('reports/{}'.format(files[i]), sep='\t')
        col1 = df.columns[0]
        col2 = df.columns[1]

        if col1 == 'date':
            df['date'] = pd.to_datetime(df['date'])

            # create a new plot with a datetime axis type
            p = figure(width=800, height=350, x_axis_type="datetime")

            # create 30 day rolling average if plotting dates
            window_size = 30
            window = np.ones(window_size)/float(window_size)
            avg = np.convolve(df[col2], window, 'same')

            source = ColumnDataSource(df)
            #add renderers
            p.line(source.data[col1], avg, color='navy', legend='avg')
            p.circle(source.data[col1], source.data[col2], size=4, color='darkgrey', alpha=0.2, legend='{}'.format(names[i]))

            p.title.text = "One-Month Average of {}".format(names[i])

        else:

            p = figure(width=800, height=350)
            source = ColumnDataSource(df)
            #add renderers
            p.line(source.data[col1], source.data[col2], line_width=2)
            p.circle(source.data[col1], source.data[col2], fill_color="white", size=8)
            p.title.text = "Count of {}".format(names[i])

        # NEW: customize by setting attributes
        p.legend.location = "top_left"
        p.grid.grid_line_alpha=0
        p.xaxis.axis_label = '{}'.format(col1)
        p.yaxis.axis_label = '{}'.format(names[i])
        p.ygrid.band_fill_color="olive"
        p.ygrid.band_fill_alpha = 0.1

        columns = [
            TableColumn(field=c, title=c, formatter=DateFormatter(format='m/d/yy') if np.issubdtype(df[c].dtype, np.datetime64) else None) for c in df.columns
            ]

        data_table = DataTable(source=source, columns=columns, width=400, height=280)

        div = Div(text="""<h2>Quick Info:</h2>
                            <ul>
                            <li>Number of observations: {}</li>
                            <li>Max: {}</li>
                            <li>Min: {}</li>
                            </ul>
                        """.format(df.shape[0], df[col2].max(), df[col2].min()),width=400, height=100)

        info = row(data_table, div) 

        tab_list.append(Panel(child= column(p, info), title=names[i]))

    return tab_list

tabs = Tabs(tabs=create_panels(files, names), width=800)


# # Set up widgets

text = TextInput(title="Title", value='One-Month Average')
windowsize = Slider(title="Window Size", value=30, start=0, end=60, step=2)

# Set up callbacks
def update_title(attrname, old, new):
    p1.title.text = text.value

def update_window(attrname, old, new):

    # Get the current slider values
    window_size = windowsize.value

    # re calculate average and re-draw line
    window = np.ones(window_size)/float(window_size)
    avg = np.convolve(source.data[col2], window, 'same')
    p1.line(source.data[col1], avg, color='navy', legend='avg')
#---

original_source = ColumnDataSource(df)


combined_callback_code = """
var data = source.get('data');
var original_data = original_source.get('data');
var grp = grp_select_obj.get('value');
console.log("grp: " + grp);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['grp'].length; ++i) {
        if (original_data['col2'][i] > grp) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.trigger('change');
source.trigger('change');
"""

# now define the callback objects now that the filter widgets exist
generic_callback = CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              grp_select_obj=windowsize, 
#              year_select_obj=year_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
##windowsize.callback = generic_callback
#

#---
#checkbox_group = CheckboxGroup(
#        labels=["Option 1", "Option 2", "Option 3"], active=[0, 1])
#button = Button(label="Foo", button_type="success")

text.on_change('value', update_title)
windowsize.on_change('value', update_window)


#tabs = Tabs(tabs=create_panels(files, names), width=800)
#inputs = widgetbox(text, windowsize, checkbox_group, button)#, amplitude, phase, freq)
inputs = widgetbox(text, windowsize)#, amplitude, phase, freq)


l= layout([inputs, tabs])
show(l)

#curdoc().add_root(row(inputs, tabs))
#curdoc().title = "Dashboard"


In [ ]:
# https://github.com/patricknieto/dashboard/blob/master/test2.py
# Cool dashboard
# Interactions not working
import pandas as pd
import numpy as np

from bokeh.io import curdoc, output_file, show
from bokeh.layouts import row, column, widgetbox, layout
from bokeh.models import ColumnDataSource, DateFormatter
from bokeh.models.widgets import Slider, TextInput, Panel, Tabs, CheckboxGroup, Div
from bokeh.models.widgets import Toggle, DataTable, DateFormatter, TableColumn, Button
from bokeh.plotting import figure,output_notebook,output_file, output_server, Session



session = Session(root_url='http://localhos:5006/', load_from_config=False)
output_server('http://localhost:5006')

files = ['urlvisits-20160823.csv', 'searches-20160823.csv', 'uniqueusers-20160823.csv', 'intab-20160823.csv']
names = ['URL Visits', 'Searches', 'Unique Users', 'In-Tab Users']


def create_panels(files, names):

	tab_list = []

	for i in range(len(files)):

		df = pd.read_csv('reports/{}'.format(files[i]),sep="\t")
		col1 = df.columns[0]
		col2 = df.columns[1]

		if col1 == 'date':
			df['date'] = pd.to_datetime(df['date'])

			# create a new plot with a datetime axis type
			p = figure(width=800, height=350, x_axis_type="datetime")

			# create 30 day rolling average if plotting dates
			window_size = 30
			window = np.ones(window_size)/float(window_size)
			avg = np.convolve(df[col2], window, 'same')

			source = ColumnDataSource(df)
			#add renderers
			p.line(source.data[col1], avg, color='navy', legend='avg')
			p.circle(source.data[col1], source.data[col2], size=4, color='darkgrey', alpha=0.2, legend='{}'.format(names[i]))
			
			p.title.text = "One-Month Average of {}".format(names[i])

		else:

			p = figure(width=800, height=350)
			source = ColumnDataSource(df)
		    #add renderers
			p.line(source.data[col1], source.data[col2], line_width=2)
			p.circle(source.data[col1], source.data[col2], fill_color="white", size=8)
			p.title.text = "Count of {}".format(names[i])

		# NEW: customize by setting attributes
		p.legend.location = "top_left"
		p.grid.grid_line_alpha=0
		p.xaxis.axis_label = '{}'.format(col1)
		p.yaxis.axis_label = '{}'.format(names[i])
		p.ygrid.band_fill_color="olive"
		p.ygrid.band_fill_alpha = 0.1

		columns = [
	        TableColumn(field=c, title=c, formatter=DateFormatter(format='m/d/yy') if np.issubdtype(df[c].dtype, np.datetime64) else None) for c in df.columns
	        ]

		data_table = DataTable(source=source, columns=columns, width=400, height=280)

		div = Div(text="""<h2>Quick Info:</h2>
							<ul>
							<li>Number of observations: {}</li>
							<li>Max: {}</li>
							<li>Min: {}</li>
							</ul>
							""".format(df.shape[0], df[col2].max(), df[col2].min()),
					width=400, height=100)

		info = row(data_table, div) 

		tab_list.append(Panel(child= column(p, info), title=names[i]))

	return tab_list


# # Set up widgets
text = TextInput(title="Title", value='One-Month Average')
windowsize = Slider(title="Window Size", value=30, start=0, end=60, step=2)

# Set up callbacks
def update_title(attrname, old, new):
    p1.title.text = text.value

def update_window(attrname, old, new):

    # Get the current slider values
    window_size = windowsize.value

    # re calculate average and re-draw line
    window = np.ones(window_size)/float(window_size)
    avg = np.convolve(source.data[col2], window, 'same')
    p1.line(source.data[col1], avg, color='navy', legend='avg')

checkbox_group = CheckboxGroup(
        labels=["Option 1", "Option 2", "Option 3"], active=[0, 1])
button = Button(label="Foo", button_type="success")

text.on_change('value', update_title)
windowsize.on_change('value', update_window)


tabs = Tabs(tabs=create_panels(files, names), width=800)
inputs = widgetbox(text, windowsize, checkbox_group, button)#, amplitude, phase, freq)


curdoc().add_root(row(inputs, tabs))
curdoc().title = "Dashboard"
#l  =layout([inputs,tabs])
#show(l)

In [ ]:
window_size= 10
window = np.ones(window_size)/float(window_size)
avg = np.convolve(df['item2'], window, 'same')
p = figure(width=800, height=350)

p.line(df['dates'], avg, color='navy', legend='avg')
output_notebook()
show(p)